In [ ]:
import numpy as np
from os.path import join
import glob
import pandas as pd
import xarray as xr
import os
import matplotlib.pyplot as plt

In [ ]:
# log-linear integration from FIAT
def get_logcov(rp_lst):
    f_lst = [1 / i for i in rp_lst]
    lf = [np.log(1 / i) for i in rp_lst]
    c = [(1 / (lf[i] - lf[i+1])) for i in range(len(rp_lst[:-1]))]
    G = [(f_lst[i] * lf[i] - f_lst[i]) for i in range(len(rp_lst))]
    a = [((1 + c[i] * lf[i+1]) * (f_lst[i] - f_lst[i+1]) + c[i] * (G[i+1] - G[i])) for i in range(len(rp_lst[:-1]))]
    b = [(c[i] * (G[i] - G[i+1] + lf[i+1] * (f_lst[i+1] - f_lst[i]))) for i in range(len(rp_lst[:-1]))]
    if len(rp_lst) == 1:
        cov_lst = f_lst
    else:
        cov_lst = [b[0] if i == 0 else f_lst[i] + a[i-1] if i == len(rp_lst) - 1 else a[i-1] + b[i] for i in range(len(rp_lst))]

    return cov_lst

def loglin_trapz(exp, rp_lst=None, nyears=None, max_rp=None):
    if rp_lst is None:
        if nyears is None:
            nyears = exp.size
        rp_lst = (1/(1-np.arange(exp.size)/exp.size)*(nyears/exp.size))
        # import pdb; pdb.set_trace()
    exp_lst = np.sort(exp)
    cov_lst = np.asarray(get_logcov(rp_lst.tolist()))
    if max_rp:
        exp_lst = exp_lst[rp_lst<=max_rp]
        cov_lst = cov_lst[rp_lst<=max_rp]
    risk = (cov_lst * exp_lst).sum()
    return risk

## read return values and simulated impact data

In [ ]:
rdir = r"../../4_results"
rm = {'h_tsw':'h'}


In [ ]:
RPS = [0,2,5,10,50,100,500]
dvars = ['qb', 'qp', 'h', 'p']
df_rps = pd.read_csv(join(rdir, f'marginal_rps.csv'), index_col=0).rename(columns=rm)
df_rps.loc[0,:] = df_rps.loc[1,:]
df_rps = df_rps.loc[RPS, dvars]
df_rps.index.name = 'rps'
df_rps

In [ ]:
index_cols = ['h_rp','p_rp','qb_rp','qp_rp']
drop_cols = ['scen','finished']#, 'h','p','qb','qp']
exp_cols = {'buildings_value': 'dam', 'population_count': 'pop'}

# read impacts and convert to 4D xarray
impacts = []
psfs = ['', '_dt0']
for i, postfix in enumerate(psfs):
    fn_impact = join(rdir, f'impact_bias0{postfix}.csv')
    df_impact = pd.read_csv(fn_impact, index_col=0).rename(columns={'h_tsw_rp':'h_rp'})
    # df_flood[index_cols] = np.maximum(1, df_flood[index_cols])
    # df_flood = df_impact.reset_index(drop=True)
    ds_impact0 = df_impact.reset_index(drop=True).drop(columns=index_cols+drop_cols).set_index(
        pd.MultiIndex.from_frame(df_impact[index_cols])
    ).to_xarray()#.fillna(0)  # fillna required to avoid nans as xarray looks at neighbors even for exact simulated locations
    # set return values on axis
    for dvar in dvars:
        ds_impact0[dvar] = xr.Variable(f'{dvar}_rp', df_rps[dvar].values)
    ds_impact = ds_impact0.swap_dims({f'{dvar}_rp': dvar for dvar in df_rps.columns})
    impacts.append(ds_impact)

In [ ]:
samples = dict(
    # indep0 = pd.read_csv(join(rdir, r'sim_EVENTS_indep_rp.csv'), index_col=0).rename(columns=rm),
    # indep = pd.read_csv(join(rdir, r'sim_EVENTS_indep1.csv'), index_col=0).rename(columns=rm),
    # fulldep = pd.read_csv(join(rdir, r'sim_EVENTS_fulldep1_rp.csv'), index_col=0).rename(columns=rm),
    # obs0 = pd.read_csv(join(rdir, r'sim_EVENTS.csv'), index_col=0).rename(columns=rm),
    indep = pd.read_csv(join(rdir, r'sim_EVENTS_indep1_30k.csv'), index_col=0).rename(columns=rm),
    fulldep = pd.read_csv(join(rdir, r'sim_EVENTS_fulldep1_30k.csv'), index_col=0).rename(columns=rm),
    obs = pd.read_csv(join(rdir, r'sim_EVENTS_30k.csv'), index_col=0).rename(columns=rm),
    # obs_lag2d = pd.read_csv(join(rdir, r'sim_EVENTS_30k_lag2dx.csv'), index_col=0).rename(columns=rm),
    # obs_lag5d = pd.read_csv(join(rdir, r'sim_EVENTS_30k_lag5dx.csv'), index_col=0).rename(columns=rm),
)
for postfix in psfs[1:]:
    for key in ['indep', 'fulldep', 'obs']:
        samples[f'{key}{postfix}'] = samples[key].copy(deep=True)
for key in samples:
    df0  = samples[key].copy()
    for dvar in dvars:
        df0[dvar] = np.minimum(np.maximum(df0[dvar], df_rps.loc[0,dvar]), df_rps.loc[500,dvar])
    ds0 = df0[dvars + ['year']].to_xarray().set_coords('year')
    postfix = '_' + key.split('_')[-1]
    if postfix in psfs[1:]:
        ds_impact = impacts[psfs.index(postfix)]
    else:
        ds_impact = impacts[0]
    ds0_impact = ds_impact.interp(ds0, method='linear')
    for col in ds_impact.data_vars.keys():
        assert not np.isnan(ds0_impact[col]).any()
        samples[key][col] = ds0_impact[col].to_series()#.fillna(0)

samples.keys()


In [ ]:
dvar = 'h'
_, ax = plt.subplots(1,1)

for key in samples:
    df0 =samples[key][dvar].copy(deep=True).sort_values().to_frame()#.plot()
    n, nyears = df0.index.size, samples[key]['year'].max()
    df0['rp'] = (1/(1-np.arange(n)/n)*(nyears/n))
    ax.plot(df0['rp'], df0[dvar], label=key)


ax.set_xlim([0,5000])
ax.set_ylim([df0.loc[df0['rp']>1, dvar].values[0], 10])
ax.grid()
ax.legend()

In [ ]:
# df0  = samples['indep'].copy()
# for dvar in dvars:
#     df0[dvar] = np.minimum(np.maximum(df0[dvar], df_rps.loc[0,dvar]), df_rps.loc[500,dvar])
# ds0 = df0[dvars + ['year']].to_xarray().set_coords('year')
# for dvar in dvars:
#     samples[dvar] = df0.copy(deep=True)
#     ds0_impact = ds_impact.isel(**{d: 0 for d in dvars if d != dvar}
#         ).interp({dvar: ds0[dvar].groupby('year').max()}, method='linear')
#     for col in ds_impact.data_vars.keys():
#         assert not np.isnan(ds0_impact[col]).any()
#         samples[dvar][col] = ds0_impact[col].to_series()#.fillna(0)

In [ ]:

samples_uni = {
    'h': ds_impact0.sel(p_rp=0, qb_rp=0, qp_rp=0).reset_coords(drop=True).to_dataframe(),
    'p': ds_impact0.sel(h_rp=0, qb_rp=0, qp_rp=0).reset_coords(drop=True).to_dataframe(),
    'qp': ds_impact0.sel(h_rp=0, qb_rp=0, p_rp=0).reset_coords(drop=True).to_dataframe(),
    'qb': ds_impact0.sel(h_rp=0, qp_rp=0, p_rp=0).reset_coords(drop=True).to_dataframe()
}

ds_fulldep = xr.merge([xr.DataArray(dims='rps', name=dvar, data=ds_impact0[dvar].values) for dvar in ds_impact0.coords])
# samples_uni['fulldep0'] =  ds_impact0.sel(ds_fulldep).to_dataframe().set_index('h_rp')#[['dam', 'ppl']]

for key in samples_uni:
    samples_uni[key].index.name = 'rp'
    samples_uni[key].index = np.maximum(1,samples_uni[key].index.values)

samples_uni['h']


In [ ]:
from copy import deepcopy
risk = {str(col): {} for col in next(iter(samples.values())).columns.values if col[:3] in ['dam', 'ppl']}
dfs = deepcopy(risk)

# univariate and full dep
for key in samples_uni:
    for exp in ['dam', 'ppl']: #risk.keys():
        df0 = samples_uni[key][[exp]]
        risk[exp][key] = loglin_trapz(df0[exp].values, df0.index.values)
    
# obs dep
nyears=1000
m = 30e3//(nyears)
for key in list(samples.keys()):
    exps = list(risk.keys()) if key == 'obs' else ['dam', 'ppl']
    for exp in exps:
        df0 = samples[key][['year', exp]].groupby('year').sum().reset_index()
        # df0['mil'] =  (df0.index.values//nyears + 1).astype(int)
        df0['mil'] = ((df0['year']-1)//nyears+1).astype(int)
        dfs[exp][key] = pd.DataFrame({i: np.sort(grp[exp].values) for i,grp in df0[df0['mil']<=m][['mil', exp]].groupby('mil')})
        risk[exp][f'{key}'] = loglin_trapz(df0.loc[df0['year']<=10000, exp].values, nyears=10000)
        out = []
        for i, df0_mil in df0.groupby('mil'):
            if np.unique(df0_mil['year']).size != nyears: 
                break
            out.append(loglin_trapz(df0_mil[exp].values, nyears=nyears))
        out = np.asarray(out)
        risk[exp][f'{key}_p05'], risk[exp][f'{key}_med'], risk[exp][f'{key}_p95'] = np.percentile(out, [5,50,95])
        risk[exp][f'{key}_min'], risk[exp][f'{key}_max'], risk[exp][f'{key}_mean'] = out.min(), out.max(), out.mean()
    #     break
    # break
df_risk = pd.DataFrame(risk)
df_risk

In [ ]:
exps = ['dam', 'ppl']
df_risk.loc[['indep_p05', 'indep_med', 'indep_p95', 'obs_p05', 'obs_med', 'obs_p95', 'fulldep_p05', 'fulldep_med', 'fulldep_p95'], exps] #- df_risk.loc['indep', exps]

In [ ]:
(df_risk.loc['fulldep_med', exps]-df_risk.loc['indep_med', exps]) / df_risk.loc['obs_med', exps] * 100

In [ ]:
df_risk.loc['obs_dt0_med', exps] / df_risk.loc['obs_med', exps] * 100


In [ ]:
labs = {
    'indep': 'independence',
    'fulldep': 'full dependence',
    # 'fulldep0': 'full dependence',
    'obs': 'obs. dependence',
    # 'magInd_timeInd': 'full indep.',
    # 'magInd_timeObs': 'independence',
    # 'magObs_timeObs_sum': 'obs. dependence',
    # 'magInd_timeInd_sum': 'full indep.',
    # 'magInd_timeObs_sum': 'independence',
    'p': 'pluvial', 
    'qp': 'fluvial Pungwe', 
    'qb': 'fluvial Buzi', 
    'h': 'coastal', 
}
rps= [1,2,5,10,25,50,100,250,500]
ylabs = {
    'dam': 'annual building damage [mil. USD]',
    'ppl': 'annual people exposed [x 1000]'
}
legend_titles = {
    'dam': '(EAD [mil. USD])',
    'ppl': '(EAAP [x 1000])'
}

In [ ]:
from matplotlib import colors, patheffects
ann_kwargs = dict(
    xytext=(0, 0),
    textcoords="offset points",
    zorder=4,
    path_effects=[
        patheffects.Stroke(linewidth=3, foreground="w"),
        patheffects.Normal(),
    ],
)

levels = ['low', 'middle', 'high']
exp0, factor = 'ppl', 1e3
# exp0, factor = 'dam', 1e6
exps = [d for d in df_risk.columns if d[:3] == exp0]
drr_dict = {}
for stat in ['med', 'p05', 'p95']:
    df_drr = df_risk.loc[f'obs_{stat}', exps].to_frame() / factor
    base, df_drr = df_drr.iloc[0], df_drr.iloc[1:]
    drr = [n.split('_')[1] for n in df_drr.index ]
    df_drr['lvl'], df_drr['drr'] = zip(*[(int(''.join([s for s in drr if s.isdigit()])), ''.join([s for s in drr if not s.isdigit()])) for drr in drr])
    df_drr['level'] = np.tile(levels, (3))
    df_drr1 = df_drr.set_index(['drr', 'level'], drop=True)[f'obs_{stat}'].unstack().loc[['zoning', 'dryproof', 'dikes'], levels]
    drr_dict[stat] = df_drr1
yerr = np.stack([(drr_dict['p95']-drr_dict['med']).values, (drr_dict['med']-drr_dict['p05']).values]).swapaxes(0,1)
ax = drr_dict['med'].plot.bar(rot=0, width=0.8, yerr=yerr, error_kw=dict(lw=2, capsize=3, capthick=1))
ax1 = ax.twinx()
ymin,ymax = ax.get_ylim()
ax1.set_ylim([0, ymax/base.item()*100])
ax.set_ylabel('expected '+ylabs[exp0])
ax.set_xlabel('')
ax1.set_ylabel('percentage of base risk [%]')
ax.legend(title='protection level')

for i, p in enumerate(ax.patches):
    j,k = i // 3, i % 3
    lvl, drr = df_drr.iloc[j+(k*3)]['lvl'], df_drr.iloc[j+(k*3)]['drr']
    txt = f'rp {lvl}' if drr != 'dryproof' else f'{lvl} cm'
    ax.annotate(txt, (p.get_x() + p.get_width()*0.25, base.item()*0.01), rotation=90, **ann_kwargs)

plt.savefig(join(rdir, f'drr_{exp0}{postfix}.png'), dpi=300, bbox_inches='tight')

In [ ]:
(1 - df_drr1 / base.item()).round(3)*100

In [ ]:
(base.item() - df_drr1).round(3)

In [ ]:
# fig, ax = plt.subplots(1,1, figsize=(6,4))
# # exp0, factor = 'ppl', 1e3
# exp0, factor = 'dam', 1e6
# key = 'obs'
# for drr in [f'{exp0}_zoning005', f'{exp0}_dryproof100', f'{exp0}_dikes010']:
#     risk0 = df_risk.loc[f'{key}_med', drr]/factor
#     lab = f'{drr} ({risk0:.2f})'
#     ls = '-' if 'obs' in key else '--'
#     lw = 1.5 if key == 'obs' else 1.5
#     df0 = dfs[drr][key].quantile([0.05,0.5,0.95], axis=1).T/factor
#     n = df0.index.size
#     df0['rp'] = (1/(1-np.arange(n)/n)*(nyears/n))
#     df0 = df0.set_index('rp')
#     ax.fill_between(df0.index.values, df0[0.05], df0[0.95], alpha=0.5, zorder=0)
#     df0[0.5].plot(ls=ls, lw=lw, ax=ax, label=lab)
# ax.set_xlim([1, 500])
# if 'dam' in exp0:
#     ax.set_ylim([0, 350])
# else:
#     ax.set_ylim([0, 150])
# ax.set_xscale('log')
# ax.set_xticks(rps)
# ax.set_xticklabels(rps)
# ax.set_ylabel(f'{ylabs[exp0]}')
# ax.set_xlabel('return period [year]')
# legend = ax.legend(title=f'compound scenario\n {legend_titles[exp0]}', loc='upper left')#, fontsize='large')
# ax.set_title('Compound flood risk - adaptation scenario')

In [ ]:
fig, ax = plt.subplots(1,1, figsize=(6,4))
exp0, factor = 'ppl', 1e3
exp0, factor = 'dam', 1e6
ls = '-'
lw = 2.5
for key in ['fulldep', 'indep', 'obs']:
    risk0 = df_risk.loc[f'{key}_med', exp0]/factor
    lab = f'{labs[key]} ({risk0:.2f})'
    df0 = dfs[exp0][key].quantile([0.05,0.5,0.95], axis=1).T/factor
    n = df0.index.size
    df0['rp'] = (1/(1-np.arange(n)/n)*(nyears/n))
    df0 = df0.set_index('rp')
    ax.fill_between(df0.index.values, df0[0.05], df0[0.95], alpha=0.3, zorder=0)#, label='0.05-095 quantile')
    df0[0.5].plot(ls=ls, lw=lw, ax=ax, label=lab, zorder=1)
ax.set_xlim([1, 500])
if 'dam' in exp0:
    ax.set_ylim([0, 550])
else:
    ax.set_ylim([0, 250])
ax.set_xscale('log')
ax.set_xticks(rps)
ax.set_xticklabels(rps)
ax.set_ylabel(f'{ylabs[exp0]}')
ax.set_xlabel('return period [year]')
legend = ax.legend(title=f'compound scenario\n {legend_titles[exp0]}', loc='upper left')#, fontsize='large')
ax.set_title('Compound flood risk - base scenario')
# ax.set_yticks(np.arange(0,500,10))
# ax.grid()
plt.savefig(join(rdir, f'risk_curve_{exp0}_compound{postfix}.png'), dpi=300, bbox_inches='tight')

In [ ]:
fig, ax = plt.subplots(1,1, figsize=(6,4))
# exp0, factor = 'ppl', 1e3
# exp0, factor = 'dam', 1e6
for key in ['qp', 'qb', 'h', 'p']:
    risk0 = df_risk.loc[key, exp0]/factor
    lab = f'{labs[key]} ({risk0:.2f})'
    df0 = samples_uni[key].copy()
    (df0[exp0]/factor).plot(ls=ls, lw=lw, ax=ax, label=lab)
ax.set_xlim([1, 500])
if 'dam' in exp0:
    ax.set_ylim([0, 550])
else:
    ax.set_ylim([0, 250])
ax.set_xscale('log')
ax.set_xticks(rps)
ax.set_xticklabels(rps)
ax.set_ylabel(f'{ylabs[exp0]}')
ax.set_xlabel('return period [year]')
legend = ax.legend(title=f'single driver scenario\n {legend_titles[exp0]}', loc='upper left')#, fontsize='large')
ax.set_title('Univariate flood risk - base scenario')
plt.savefig(join(rdir, f'risk_curve_{exp0}_single{postfix}.png'), dpi=300, bbox_inches='tight')
